In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from dataset import LandscapeSimulationDataset
from model import PhiNN
from model_training import train_model
from helpers import select_device, jump_function, mean_cov_loss

In [3]:
outdir = "out/testing"
dtype = torch.float32
device = select_device()
print(f"Using device: {device}")

Using device: cpu


In [4]:
datdir_train = "data/model_training_data"
datdir_valid = "data/model_validation_data"

nsims_train = 2
nsims_valid = 1
ndims = 2
nsigs = 2
ncells = 100
dt = 1e-3
sigma = 1e-3

batch_size = 2

In [5]:
train_dataset = LandscapeSimulationDataset(
    datdir_train, nsims_train, ndims, 
    transform='tensor', 
    target_transform='tensor',
    dtype=dtype,
)

validation_dataset = LandscapeSimulationDataset(
    datdir_valid, nsims_valid, ndims, 
    transform='tensor', 
    target_transform='tensor',
    dtype=dtype,
)

train_dataloader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True
)

validation_dataloader = DataLoader(
    validation_dataset, 
    batch_size=batch_size, 
    shuffle=False
)

print("Training Dataset Length:", len(train_dataset))
print("Validation Dataset Length:", len(validation_dataset))
print("Training DataLoader Length:", len(train_dataloader))
print("Validation DataLoader Length:", len(validation_dataloader))

Training Dataset Length: 200
Validation Dataset Length: 100
Training DataLoader Length: 100
Validation DataLoader Length: 50


In [6]:
# Construct the model
f_signal = lambda t, p: jump_function(t, p[...,0], p[...,1:3], p[...,3:])
model = PhiNN(
    ndim=ndims, nsig=nsigs, f_signal=f_signal, 
    ncells=ncells, 
    sigma=sigma,
    device=device,
    dtype=dtype,
).to(device)

loss_fn = mean_cov_loss

learning_rate = 1e-2
momentum = 0.9

optimizer = torch.optim.SGD(
    model.parameters(), 
    lr=learning_rate, 
    momentum=momentum
)

In [7]:
input, expected = next(iter(train_dataloader))

In [10]:
print(input.shape)
print(expected.shape)

torch.Size([2, 207])
torch.Size([2, 100, 2])


In [11]:
from torch.profiler import profile, record_function, ProfilerActivity

In [12]:
with profile(activities=[ProfilerActivity.CPU], 
             profile_memory=True, record_shapes=True) as prof:
    model(input)
    # train_model(
    #     model, dt, loss_fn, optimizer, 
    #     train_dataloader, validation_dataloader,
    #     num_epochs=1,
    #     batch_size=batch_size,
    #     device=device,
    #     model_name='testmodel',
    #     outdir=outdir,
    # )

[W kineto_shim.cpp:333] Adding profiling metadata requires using torch.profiler with Kineto support (USE_KINETO=1)
[W kineto_shim.cpp:333] Adding profiling metadata requires using torch.profiler with Kineto support (USE_KINETO=1)


In [13]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               aten::mm        21.16%      49.006ms        22.18%      51.366ms      42.805us       7.48 Mb       7.40 Mb          1200  
                                            aten::addmm         8.79%      20.360ms        11.84%      27.432ms      27.432us       7.40 Mb       7.40 Mb          1000  
                                         aten::softplus         8.67%      20.079ms         8.67%      20.079ms      25.099us       7.32 Mb       7.32

In [14]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               aten::mm        21.16%      49.006ms        22.18%      51.366ms      42.805us       7.48 Mb       7.40 Mb          1200  
    autograd::engine::evaluate_function: AddmmBackward0         1.17%       2.707ms        17.13%      39.682ms      39.682us       7.48 Mb     -32.03 Kb          1000  
                                         AddmmBackward0         2.17%       5.037ms        15.94%      36.912ms      36.912us       7.48 Mb      32.03